# Run functions from QCrBoxTools from QCrBox
QCrBoxTools is a library that is installed in any QCrBox container. At the moment it contains three types of functionality. Firstly, functionality to modify, trim, merge or convert the cif entries if crystallographic information framework (cif) files. Secondly, implementations to automatically run some programs we have added to the QCrBox (which are the "Robot" classes.) Finally, functions which are needed in a pipeline workflow of QCrBox. The latter functionality is exposed via the QCrBoxTools container.

The example starts again by using the dotenv package, as it makes things more convenient because we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, comment out the load_dotenv lines and insert the local and qcrbox pathes manually into the pathhelper from the .env.dev file.

In [ ]:
import os
import shutil
import json

from dotenv import load_dotenv

from qcrbox_wrapper import QCrBoxWrapper, QCrBoxPathHelper


if not load_dotenv('../.env.dev'):
    raise FileNotFoundError(
        ".dot.env file could not be loaded. Either adapt the path to your filesystem or "
        + "input the information loaded from os.environ manually"
    )


We create an example folder to try out this functionality.

In [ ]:
pathhelper = QCrBoxPathHelper(
    os.environ['QCRBOX_SHARED_FILES_DIR_HOST_PATH'],
    os.environ['QCRBOX_SHARED_FILES_DIR_CONTAINER_PATH'],
    'examples_qcrboxtools'
)

path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

## Connecting to QCrBox
We can connect to the QCrBox Inventory via python after we have started everything with qcb up. Sometimes the server takes a while so you might need to retry if it initially refuses connection. This should not take more than 30 seconds after your console output says that everything has started.

In [ ]:
qcrbox = QCrBoxWrapper('localhost', 11000)

In [ ]:
qcrboxtools = qcrbox.application_dict['QCrBoxTools']

In [ ]:
help(qcrboxtools)

## Converting a given cif to the unified keywords used in QCrBoxTools

If we start with a given cif and want to create a unified cif. QCrBox works with a subsect of cif entries, which are the base cif entries in the current cif dictionaries (as of February 2024). Additionally uncertainties are split into their own entries. To convert any cif file into this format we can use the `to_unified_cif` method of QCrBoxTools. If you prefer to use an outside library, the QCrBoxtools python library also offers this functionality in the `qcrboxtools.cif.cif2cif` module.

Let us copy an example file and do the transformation.

In [ ]:
folder_unify = path_local / 'run_unify_cif'
folder_unify.mkdir(exist_ok=True)

shutil.copy('./input_files/non_unified_kws.cif', folder_unify / 'non_unified_kws.cif')

calc4 = qcrboxtools.to_unified_cif(
    input_cif_path=path_qcrbox / 'run_unify_cif' / 'non_unified_kws.cif',
    output_cif_path=path_qcrbox / 'run_unify_cif' / 'output.cif',
    custom_category_list='iucr olex shelx'
)

print(calc4.status)
calc4.wait_while_running(0.2)
print(calc4.status)

In [ ]:
calc = qcrboxtools.check_structure_convergence(
    cif1_path='/mnt/qcrbox/shared_files/test_convergence_func/difference_test1.cif',
    cif2_path='/mnt/qcrbox/shared_files/test_convergence_func/difference_test2.cif',
    max_abs_position='0.001',
    max_position_su='None',
    max_abs_uij='0.005',
    max_uij_su='1.0',
    output_json='/mnt/qcrbox/shared_files/test_convergence_func/output.json'
)

calc.wait_while_running(0.2)
print(calc.status)

In [ ]:
calc2 = qcrboxtools.iso2aniso(
    cif_path='/mnt/qcrbox/shared_files/test_iso2aniso/olex_unified.cif',
    cif_dataset='0',
    select_names='None',
    select_elements='H',
    select_regexes='None'
)

In [ ]:
print(calc2.status)
calc2.wait_while_running(0.2)
print(calc2.status)

In [ ]:
calc3 = qcrboxtools.replace_structure_from_cif(
    input_cif_path='/mnt/qcrbox/shared_files/replace_test/test.cif',
    structure_cif_path='/mnt/qcrbox/shared_files/replace_test/80K_P_out.cif'
)

In [ ]:
print(calc3.status)
calc3.wait_while_running(0.2)
print(calc3.status)